In [1]:
import sys
print(sys.executable) 

d:\ProgramData\anaconda3\python.exe


In [1]:
# test_env.py
import cv2
import numpy as np
import win32gui
import torch
from ultralytics import YOLO

print("OpenCV version:", cv2.__version__)
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())

OpenCV version: 4.10.0
PyTorch version: 2.5.1+cpu
CUDA available: False


In [2]:
import torch
print(torch.__version__) 
print(torch.cuda.is_available()) 

2.5.1+cpu
False


In [1]:
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

PyTorch version: 2.5.1+cu121
CUDA available: True
CUDA device: NVIDIA GeForce RTX 4060 Laptop GPU
CUDA version: 12.1


In [2]:
from ultralytics import YOLO
model = YOLO('./boss_plus_wukong.pt')  


print("architecture:")
print(model.model)


print("\nclass:")
print(model.names)


print("\nparam:")
for param, value in model.overrides.items():
    print(f"{param}: {value}")


architecture:
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inpl

In [4]:
from ultralytics import YOLO

model = YOLO('./boss_only.pt')  


print("模型架构:")
print(model.model)

print("\n类别信息:")
print(model.names)

print("\n超参数:")
for param, value in model.overrides.items():
    print(f"{param}: {value}")


模型架构:
DetectionModel(
  (model): Sequential(
    (0): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (1): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (2): C2f(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (cv2): Conv(
        (conv): Conv2d(320, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True

In [1]:
from ultralytics import YOLO
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt


action_labels = {
    0: 'boss_jump_attack', 
    1: 'boss_feet_attack', 
    2: 'boss_clap_attack', 
    3: 'boss_far_attack', 
    4: 'boss_boom_attack', 
    5: 'boss_sweep_attack', 
    6: 'boss_cry', 
    7: 'boss_down', 
    8: 'boss_stand',
    99: 'no_action'
}


try:
    model = YOLO("./boss_only.pt")
    print("模型加载成功！")
    
  
    model_classes = model.names
    print("\n模型类别信息:")
    print(f"预期类别数: 9")
    print(f"实际类别数: {len(model_classes)}")
    
    # 比较模型类别和预定义标签
    print("\n类别标签对比：")
    print("预定义标签 vs 模型标签")
    print("-" * 40)
    for i in range(9):  # exclude no_action
        defined_label = action_labels[i]
        model_label = model_classes[i]
        match = "✓" if defined_label == model_label else "✗"
        print(f"Class {i}: {defined_label:15} vs {model_label:15} {match}")
        
    # 验证模型参数
    print("\n模型配置信息:")
    print(f"任务类型: {model.task}")
    print(f"图像输入尺寸: {model.model.args['imgsz']}")
    
except Exception as e:
    print(f"错误：{str(e)}")

# test
test_image_path = "./bloodfinder.png"  

try:
    # 尝试进行一次推理
    results = model(test_image_path, conf=0.25)
    print("\n推理测试:")
    print("成功进行推理！检测到的动作：")
    
    # 显示检测结果
    for r in results:
        for box in r.boxes:
            class_id = int(box.cls[0])
            conf = float(box.conf[0])
            if class_id in action_labels:
                print(f"动作：{action_labels[class_id]}，置信度：{conf:.2f}")
                
except FileNotFoundError:
    print("\n未找到测试图像,跳过推理测试")
except Exception as e:
    print(f"\n推理测试出错:{str(e)}")

模型加载成功！

模型类别信息：
预期类别数: 9
实际类别数: 9

类别标签对比：
预定义标签 vs 模型标签
----------------------------------------
Class 0: boss_jump_attack vs boss_jump_attack ✓
Class 1: boss_feet_attack vs boss_feet_attack ✓
Class 2: boss_clap_attack vs boss_clap_attack ✓
Class 3: boss_far_attack vs boss_far_attack ✓
Class 4: boss_boom_attack vs boss_boom_attack ✓
Class 5: boss_sweep_attack vs boss_sweep_attack ✓
Class 6: boss_cry        vs boss_cry        ✓
Class 7: boss_down       vs boss_down       ✓
Class 8: boss_stand      vs boss_stand      ✓

模型配置信息：
任务类型: detect
图像输入尺寸: 640

image 1/1 d:\learn\2024fall\DS\finalproject\YOLOPlusDQN\bloodfinder.png: 384x640 (no detections), 40.5ms
Speed: 6.2ms preprocess, 40.5ms inference, 35.7ms postprocess per image at shape (1, 3, 384, 640)

推理测试：
成功进行推理！检测到的动作：
